In [168]:
import glob
import os
import cv2
import pandas as pd
import numpy as np

import os
import cv2
import pandas as pd
import numpy as np

# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms

# from tqdm import tqdm
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 아래 코드 230721 정유라 추가함
import gc
from skimage.color import lab2lch, rgb2lab
from skimage.exposure import rescale_intensity
from skimage.morphology import disk
from sklearn.cluster import KMeans


In [169]:
def rle_decode(mask_rle, shape): # rle_디코딩
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [170]:
def cut_image(image, subimage_size=256):

    subimages = []
    for i in range(0, image.shape[0], subimage_size):
        for j in range(0, image.shape[1], subimage_size):
            subimage = image[i:i+subimage_size, j:j+subimage_size]
            subimages.append(subimage)
    return subimages

In [171]:
import os
current_dir = os.getcwd()
dat = pd.read_csv(os.path.join(current_dir, 'train.csv'))
dat.head()

,img_id,img_path,mask_rle
0,TRAIN_0000,./train_img/TRAIN_0000.png,9576 7 10590 17 11614 17 12638 17 13662 17 146...
1,TRAIN_0001,./train_img/TRAIN_0001.png,208402 1 209425 6 210449 10 211473 14 212497 1...
2,TRAIN_0002,./train_img/TRAIN_0002.png,855 34 15654 9 16678 9 16742 8 17702 9 17766 9...
3,TRAIN_0003,./train_img/TRAIN_0003.png,362 6 745 15 798 22 900 25 1385 8 1828 16 1924...
4,TRAIN_0004,./train_img/TRAIN_0004.png,34 27 1058 27 2082 27 3105 27 4129 27 5153 27 ...


In [172]:
import os

def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# save_img = './cut_img' # 자른 이미지 저장
# create_directory(save_img)

# save_msk = './cut_msk' # 자른 마스크 저장
# create_directory(save_msk)

save_img_2 = './1_cut_img'  # 자른 이미지 저장
create_directory(save_img_2)

save_msk_2 = './1_cut_msk'  # 자른 마스크 저장
create_directory(save_msk_2)

In [173]:
## 여기서 저장할 이미지 개수를 조절해요. 

# len(dat) = 7140
lens = len(dat) -7139

In [174]:
# def count_and_plot(images, bin_edges):
#     pixel_counts = []

#     # Iterate through all images in the folder
#     for image in images:
#         x, y = image.shape
#         building_pixels = (image > 0).astype(np.uint8) # <-- 바꾼 부분
#         # Count building pixels and add to the list
#         building_pixels = np.sum(image == 1)
#         pixel_counts.append(building_pixels/(x*y)*100)
#         # print(f'{building_pixels} / {x*y}')
    
#     # Sorting the pixel counts
#     pixel_counts.sort()

#     # Preparing bins for the histogram
#     bin_labels = [f'{bin_edges[i]} - {bin_edges[i+1]}' for i in range(len(bin_edges)-1)]

#     # Plotting the histogram
#     plt.figure(figsize=(10,6))
#     plt.hist(pixel_counts, bins=bin_edges, edgecolor='black', alpha=0.7)
#     plt.title("Distribution of Building Pixels Across Images")
#     plt.xlabel("Number of Building Pixels")
#     plt.ylabel("Number of Images")
#     plt.xticks(bin_edges, fontsize=1)
#     plt.show()

#     # Returning the counts as well
#     return pixel_counts

In [175]:
## 이미지, 마스크 페어로 저장 복사본

for i in range(0, lens):
    
    # 이미지 불러오기
    image = cv2.imread(dat.iloc[i,1][2:])
    # 마스크 불러오기
    mask = rle_decode(dat.iloc[i, 2], (1024, 1024))

    # 이미지를 256*256 사이즈의 서브 이미지로 분할
    sub_images = cut_image(image, 256)
    # 마스크를 256x256 사이즈의 서브 마스크로 분할
    sub_masks = cut_image(mask, 256)

    # 분할된 마스크를 png로 저장
    for idx, (submask, subimage) in enumerate(zip(sub_masks, sub_images)):
 
        white_pixels = np.count_nonzero(submask > 0)  # 이진영상에서 건물 유무 확인
    
        
        if white_pixels > 0: # 건물 영역인 픽셀의 개수가 0개 초과인 경우
            
            cv2.imwrite(f'{save_img_2}/1_TRAIN_{dat.iloc[i,0][6:]}_{str(idx+1).zfill(2)}.png', subimage)
            cv2.imwrite(f'{save_msk_2}/1_TRAIN_{dat.iloc[i,0][6:]}_{str(idx+1).zfill(2)}.png', submask) # 파일명 ex) cut_msk/1_TRAIN_0000_01

In [176]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader

import os
from PIL import Image
import torch
from torch.utils.data import Dataset

class ImageDataset(Dataset):
    def __init__(self, image_folder, mask_folder, transform=None, infer=False):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.image_filenames = os.listdir(image_folder)
        self.mask_filenames = os.listdir(mask_folder)
        self.infer = infer

        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        image_path = os.path.join(self.image_folder, self.image_filenames[index])
        mask_path = os.path.join(self.mask_folder, self.mask_filenames[index])

        image = Image.open(image_path).convert("RGB")

        if self.infer:
            if self.transform:
                image = self.transform(image)
            return image

        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

In [177]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
#     transforms.RandomHorizontalFlip(),  # 랜덤 수평 뒤집기
#     transforms.RandomVerticalFlip(),  # 랜덤 수직 뒤집기
#     transforms.RandomRotation(30),  # 랜덤 회전 (최대 30도)
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 랜덤 컬러 조정
    transforms.ToTensor(),  # 이미지를 Tensor로 변환
])

In [178]:
import segmentation_models_pytorch as smp
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

# Define the U-Net model with ResNet34 backbone
model = smp.Unet(
    encoder_name='resnet34',        # Use ResNet34 as the encoder backbone
    encoder_weights='imagenet',     # Use pretrained weights on ImageNet
    in_channels=3,                  # Input channels, adjust according to your data
    classes=1,                      # Number of output classes, adjust according to your task
)

# Move the model to the appropriate device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define other training parameters (adjust as needed)
batch_size = 16
learning_rate = 0.1
num_epochs = 1

# 데이터셋 인스턴스 생성
dataset = ImageDataset( image_folder = './1_cut_img', mask_folder = './1_cut_msk', transform=transform)

train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [179]:
def train_model(model, criterion, optimizer, dataloader, device):
    model.train()
    running_loss = 0.0

    for images, masks in dataloader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate the loss
        loss = criterion(outputs, masks)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate the average loss for this epoch
    train_loss = running_loss / len(dataloader)
    return train_loss

# Training loop
for epoch in range(num_epochs):
    train_loss = train_model(model, criterion, optimizer, train_dataloader, device)
    torch.save(model.state_dict(), f'trained_model_all_{epoch+1}.pth')
    print(f'Epoch {epoch+1}, Loss: {train_loss}')


Epoch 1, Loss: 1.7218149900436401


# 3. Evaluation

In [180]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    if len(runs) % 2 == 1:  # 짝수 개의 요소로 만들기 위해
        runs = np.append(runs, len(pixels))
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def compute_otsu_threshold(mask):
    # Convert the mask to an 8-bit single-channel image
    mask = (mask * 255).astype(np.uint8)

    # Apply Otsu's method to find the optimal threshold
    _, threshold = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return threshold / 255  # Normalize the threshold to the range [0, 1]

def evaluate_and_visualize_segmentation(model, dataloader, device):
    model.eval()
    result = []
    with torch.no_grad():
        for images, mask in dataloader:
            images = images.to(device)
            outputs = model(images)

            binary_masks = torch.sigmoid(outputs).cpu().numpy()
            binary_masks = np.squeeze(binary_masks, axis=1)
            binary_masks_orig = binary_masks

            otsu_thresholds = [compute_otsu_threshold(mask) for mask in binary_masks_orig]
            binary_masks = [(mask > threshold).astype(np.uint8) for mask, threshold in zip(binary_masks_orig, otsu_thresholds)]
            
            images_np = images.cpu().numpy()

            for i in range(len(images)): # 시각화 (이미지 개수 조정)
                mask_rle = rle_encode(binary_masks[i])
                if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                 result.append(-1)
                else:
                  result.append(mask_rle)

                plt.figure(figsize=(20, 5))
                plt.subplot(1, 3, 1)
                plt.imshow(np.transpose(images_np[i], (1, 2, 0)))
                plt.title('Input Image')

                plt.subplot(1, 3, 2)
                plt.imshow(binary_masks[i], cmap='gray')
                plt.title('Predicted Mask')

                plt.subplot(1, 3, 3)
                histogram, bins = np.histogram(255 * binary_masks_orig[i, 0][0:], bins=256, range=[0, 256])
                plt.plot(histogram, color='black')
                plt.title('Histogram of Predicted Mask')
                plt.xlim([0, 256])
    return result

In [181]:
import torch

model.load_state_dict(torch.load('trained_model_all_1.pth'))
# # model.eval()  # set the model to evaluation mode

<All keys matched successfully>

In [182]:
from torchvision.transforms import ToTensor

infer_dataset = ImageDataset(image_folder='test_img', mask_folder=None, transform=transform, infer=True)


In [183]:

infer_dataloader = DataLoader(infer_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
result = evaluate_and_visualize_segmentation(model, infer_dataloader, device)

IndexError: list index out of range

In [ ]:
import pandas as pd

submit = pd.read_csv('./submission.csv')
submit['mask_rle'] = result

submit.to_csv('./submit.csv', index=False)